# Temporal Fluctuations

Topic: periodic activity can be variable through time.

Sim: demonstrate that signals with different amounts of bursty-ness show power differences

- can show across continuous data, and also simulating 'epochs'
- can compare to power differences of continuous oscillations

Core citations / links:

- Jones lab work on this topic / link to their paper(s) 